<a href="https://colab.research.google.com/github/Roxasxxx/backup/blob/main/%E6%95%B4%E7%90%86%E6%90%8D%E7%9B%8A%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import requests
import pandas as pd
from io import StringIO
import time
import random
from google.colab import files

In [26]:
statements={'BalanceSheet': "https://mops.twse.com.tw/mops/web/ajax_t164sb03",
    'ProfitAndLose': "https://mops.twse.com.tw/mops/web/ajax_t164sb04",
    'CashFlowStatement' : "https://mops.twse.com.tw/mops/web/ajax_t164sb05"}

def get_statement(stock,year,season,statement):

    try:
        # 爬取資料
        form_data = {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'co_id':stock,
            'year': year-1911,
            'season': season,
        }

        r = requests.post(statement,form_data)

        # 使用 StringIO 來包裝 HTML 字符串
        html_io = StringIO(r.text)
        html_df = pd.read_html(html_io)[1].fillna("")

        #整理表格
        html_df.columns = html_df.columns.get_level_values(-1)
        thisyear = f'{year}Q{season}'
        lastyear = f'{year-1}Q{season}'
        html_df = html_df.drop(['%'], axis=1)
        overall_df = html_df.iloc[:, :-1]
        if season in [1, 4]:
            cols = ['account',f'{thisyear}_sum',f'{lastyear}_sum']
        else:
            cols = ['account',thisyear,lastyear,f'{thisyear}_sum',f'{lastyear}_sum']
        overall_df.columns=cols
        return overall_df

    except Exception as e:
        print(f"{stock} {year}Q{season} statement is not exist. Error: {str(e)}")
        return None


In [27]:
def create_new_df(df, accounting, thisquarter, lastquarter, name):
    new_data = {'name': name}
    try:
        for account in accounting:
            thisyear_value = df.loc[df['account'] == account, f'{thisquarter}_sum'].values[0]
            lastyear_value = df.loc[df['account'] == account, f'{lastquarter}_sum'].values[0]

            new_data[f'{thisquarter}_sum{account}'] = thisyear_value
            new_data[f'{lastquarter}_sum{account}'] = lastyear_value
    except:
        print('create error')
    return new_data

In [28]:
#定義變數
# 輸入需要的股票代號、年度、季度、會計科目

names = [2459,
3010,
3028,
3033,
3036,
3048,
3209,
3232,
3312,
3360,
3444,
3528,
3702,
5434,
6113,
6189,
6192,
6227,
6259,
6265,
6270,
8032,
8068,
8070,
8096,
8112]


year = 2024
season = 1
accounting = ['營業收入合計','營業毛利（毛損）','營業利益（損失）', '稅前淨利（淨損）', '本期淨利（淨損）']
thisquarter = f'{year}Q{season}'
lastquarter = f'{year-1}Q{season}'

In [29]:
# 構建迴圈

# 建立空Dataframe儲存資料
profit_data = pd.DataFrame()
n = 0

for name in names:
    #爬取資料
    try:
        print(name)
        df = get_statement(name,year,season,statements['ProfitAndLose'])
        new_data = pd.DataFrame([create_new_df(df,accounting,thisquarter,lastquarter,name)],index=[n+1])
        new_df = pd.DataFrame(new_data)
        new_df.head()
        profit_data = pd.concat([profit_data, new_df], ignore_index=True)
        n+=1
        #停滯時間
        if n%random.randint(3,5) == 0:
            time.sleep(random.randint(20,40))
        elif n == len(names):
          print('done')
          pass
    except:
        print(str(name)+'無法取得資料')
        pass

2459
3010
3028
3033
3036
3048
3209
3232
3312
3360
3444
3528
3702
5434
6113
6189
6192
6227
6259
6265
6270
8032
8068
8070
8096
8112
Done


In [30]:
#計算整理

for col in accounting:
    profit_data[f'{col}億元'] = profit_data[f'{thisquarter}_sum{col}'] / 100000
    profit_data[f'{col}年增率'] = pd.Series(dtype='object')
    profit_data[f'{col}年增率'] = (profit_data[f'{thisquarter}_sum{col}'] - profit_data[f'{lastquarter}_sum{col}']) / profit_data[f'{lastquarter}_sum{col}'] * 100

    # 將年增率欄位轉換為 object 型別以便能夠存儲字符串
    profit_data[f'{col}年增率'] = profit_data[f'{col}年增率'].astype(object)

for idx in profit_data.index:
    for col in accounting:
        if profit_data.loc[idx, f'{lastquarter}_sum{col}'] < 0 and profit_data.loc[idx, f'{thisquarter}_sum{col}'] > 0:
            profit_data.loc[idx, f'{col}年增率'] = '由虧轉盈'
        elif profit_data.loc[idx, f'{lastquarter}_sum{col}'] > 0 and profit_data.loc[idx, f'{thisquarter}_sum{col}'] < 0:
            profit_data.loc[idx, f'{col}年增率'] = '由盈轉虧'
        elif profit_data.loc[idx, f'{lastquarter}_sum{col}'] < profit_data.loc[idx, f'{thisquarter}_sum{col}'] < 0:
            profit_data.loc[idx, f'{col}年增率'] = '虧損減少'
        elif profit_data.loc[idx, f'{thisquarter}_sum{col}'] < profit_data.loc[idx, f'{lastquarter}_sum{col}'] < 0:
            profit_data.loc[idx, f'{col}年增率'] = '虧損增加'

In [31]:
# 生成的新列表
accounting_list = ['name']

# 迭代列表中的每一個指標
for indicator in accounting:
    accounting_list.append(f"{indicator}億元")
    accounting_list.append(f"{indicator}年增率")

In [32]:
output = profit_data[accounting_list]

In [33]:
output.to_csv(f'statement_output_{thisquarter}.csv', index=False,encoding="utf_8_sig")

In [34]:
files.download(f'statement_output_{thisquarter}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>